Notebook 1: Merging and annotating tables and matrix tables, Sample QC, PCA outlier removal, Variant QC

To do:
1. add in plotting code generated by Ally & make sure it runs
2. add more detailed descriptions of why each step is run
3. remove certain table/mt write-outs
4. potentially (need to discuss with Mary first) write a function to read data
5. create more detailed index

---------------------

Notebook 3: Sequencing/diversity and QC metrics notebook

Plots that need to be added to nb (currently in the works by Ally) 
1. Contamination/Freemix - sample_filters.contamination and bam_metrics.freemix - *DONE*
2. Heterozygosity - plot distribution to show how it doesn’t work with diverse popns as expected (e.g. high for AFR, low for FIN) - *PENDING*
3. Plot fail_n_snp_residual and using that as an example, explain the rest in the description - the plots were created early on in the project showing which gnomAD QC filters were dropping whole populations. Currently in R, but needs to be implemented in Hail - *PENDING*

---------------------
Further edits needed in this nb:
- Add Ally's code for plots 

## Index
1. [Data Management Function](#1.-Data-Management-Function)
2. [Read in Datasets and Annotate](#2.-Read-in-Datasets-and-Annotate)
3. [Investigating gnomAD sample filters](#3.-Investigating-gnomAD-sample-filters)

# General Overview
The purpose of this script is to merge metadata components needed for the HGDP+1kGP dataset and then run  QC filters on that resulting dataset. The metadata included sample and variant information such as geographic region, and which samples/variants passed QC were initially located in different datasets. The QC filters were run using sample/variant flags from the metadata datasets. These flags were generated as a result of the dataset being run through the gnomAD QC pipeline. More information on the gnomAD QC pipeline can be found [here](https://gnomad.broadinstitute.org/news/2020-10-gnomad-v3-1-new-content-methods-annotations-and-data-availability/#sample-and-variant-quality-control). To see how these filters were updated as a result of our analyses, see [gnomAD sample filters](#3.-Investigating-gnomAD-sample-filters) and the resulting gnomAD [minor release.](https://gnomad.broadinstitute.org/news/2021-10-gnomad-v3-1-2-minor-release/#improvements-to-the-hgdp--1kg-subset-release)

**This script contains information on how to**:
- Annotate new fields onto a matrix table from another matrix table or hail table
- Unflatten a hail matrix table
- Harmonize datasets to prevent merge conflicts
- Use plots to identify which gnomAD QC filters are removing populations entirely (fail_n_snp_residual used as an example)
- Retrieve populations being unduly removed by filters (mostly AFR and OCE populations)
- Filter matrix tables using a field within the matrix table
- Filter samples using a hardcoded list of samples to remove
- Plot certain fields from the matrix table:
    - Contamination
    - Freemix
    - Heterozygosity (plot distribution to show how it doesn’t work with diverse populations as expected (e.g. high for AFR, low for FIN)

**Datasets merged are**:
- sample_meta: sample metadata table which contains harmonized metadata for the HGDP_1kGP dataset
- sample_qc_meta: gnomad v3.1 sample qc metadata from for the hgdp_1kg subset which contains flags to denote which samples failed gnomAD QC filters
- var_meta: hail matrix table with a field of flags (mt.filters) to denote which variants passed or failed gnomAD qc filters
- dense_mt: densified hgdp_1kg matrix table

Authors: Zan Koenig & Mary T. Yohannes

In [1]:
# import hail
import hail as hl

# for renaming purposes
import re

## Set Requester Pays Bucket
Running through these tutorials, users must specify which project is to be billed. To change which project is billed, set the `GCP_PROJECT_NAME` variable to your own project.

In [ ]:
# setting requester pays bucket to use throughout tutorial
GCP_PROJECT_NAME = "diverse-pop-seq-ref"
hl.init(spark_conf={
    'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
    'spark.hadoop.fs.gs.requester.pays.buckets': 'hgdp_tgp,gcp-public-data--gnomad',
    'spark.hadoop.fs.gs.requester.pays.project.id': 'diverse-pop-seq-ref'
})

# 1. Data Management Function
This function serves the purpose of reading in the dataset of different stages throughout the tutorial and given certain flags, will allow the user to specify which filters they would like run on the dataset. This function helps to reduce the amount of times data needs to be written out, overall decreasing the computational and monetary cost of running the tutorials. 

<details>
    <summary><br>The arguments of the function are as follows: 
        <br>[click <span style="color:green">here</span> to expand]</summary>
<br> 
Click on each argument name to learn more!

<details><summary><br> 
- <span style="color:green">default</span></summary> 
    
<p>when <i><b>True</b></i>, will return a pre-QC matrix table</p></details>

<details><summary><br> 
- <span style="color:green">post_qc</span></summary> 
    
<p>when <i><b>True</b></i>, will return a matrix table which has the following conducted:</p>
<ul> 
    <li>sample_qc filtering</li>
    <li>variant_qc filtering</li>
    <li>outlier removal</li>
    <li>duplicate removal</li>  
    </ul></details>


<details><summary><br>   
- <span style="color:green">sample_qc</span></summary>     
    
<p>when <i><b>True</b></i>, will return a matrix table with gnomad's sampleQC filters run on the dataset. For more information on gnomAD's sample QC steps click <a href="https://gnomad.broadinstitute.org/news/2020-10-gnomad-v3-1-new-content-methods-annotations-and-data-availability/#sample-qc-hard-filtering"> here.</a></p></details>    
    
<details><summary><br>   
- <span style="color:green">variant_qc</span></summary>     
    
<p>when <i><b>True</b></i>, will return a matrix table with gnomad's variant quality control filters run on the dataset. For more information on gnomAD's variant QC steps click <a href="https://gnomad.broadinstitute.org/news/2020-10-gnomad-v3-1-new-content-methods-annotations-and-data-availability/#variant-qc"> here.</a></p></details>        

<details><summary><br>   
- <span style="color:green">duplicate</span></summary>     
    
<p>when <i><b>True</b></i>, will return a matrix table with any duplicates in the dataset removed. By default there are no duplicates in the dataset, but was included as it is a useful QC step to demonstrate</p></details> 
  
<details><summary><br>
- <span style="color:green">outlier_removal</span></summary>     
    
<p>when <i><b>True</b></i>, will return a matrix table with pca outliers removed. These outliers were determined by running pc_relate. More information on how we created the outlier list can be found <a href="https://nbviewer.org/github/atgu/hgdp_tgp/blob/master/tutorials/nb4.ipynb#5.-Outlier-Removal"> here.</a></p></details> 
    
<details><summary><br>   
- <span style="color:green">ld_pruning</span></summary>     
   
<p>when <i><b>True</b></i>, will return a matrix table which has the following conducted:</p>
<ul> 
    <li>sample_qc filtering</li>
    <li>variant_qc filtering</li>
    <li>outlier removal</li>
    <li>duplicate removal</li>
    <li>call rate filter to variants whose call rate is > 0.999</li>
    <li>allele frequency filter on variants to only keep variants with 0.05 < AF < 0.95</li>
    </ul></details>   
    
<details><summary><br>
- <span style="color:green">rel_unrel</span></summary>  

<p>when <i><b>default</b></i>, will return the same matrix table which would be returned when ld_pruning=True</p>

<p>when <i><b>related</b></i>, will return a matrix table with only related samples</p>

<p>when <i><b>unrelated</b></i> will return matrix table with only unrelated samples</p></details>

   

In [ ]:
def read_qc(
        default: bool = False,
        post_qc:bool = False,
        sample_qc: bool = False,
        variant_qc: bool = False,
        duplicate: bool = False,
        outlier_removal: bool = False,
        ld_pruning: bool = False,
        rel_unrel: str = 'default') -> hl.MatrixTable:
    """
    Wrapper function to get HGDP+1kGP data as Matrix Table at different stages of QC/filtering.
    By default, returns pre QC MatrixTable with qc filters annotated but not filtered.

    :param bool default: if True will preQC version of the dataset
    :param bool post_qc: if True will return a post QC matrix table that has gone through:
        - sample QC
        - variant QC
        - duplicate removal
        - outlier removal
    :param bool sample_qc: if True will return a post sample QC matrix table
    :param bool variant_qc: if True will return a post variant QC matrix table
    :param bool duplicate: if True will return a matrix table with duplicate samples removed
    :param bool outlier_removal: if True will return a matrix table with PCA outliers and duplicate samples removed
    :param bool ld_pruning: if True will return a matrix table that has gone through:
        - sample QC
        - variant QC
        - PCA outlier removal
        - duplicate removal
        - LD pruning
        - additional variant filtering
    :param bool rel_unrel: default will return same mt as ld pruned above
        if 'related' will return a matrix table with only related samples.
        if 'unrelated' will return a matrix table with only unrelated samples
    """
    # Reading in all the tables and matrix tables needed to generate the pre_qc matrix table
    sample_meta = hl.import_table('gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/gnomad_meta_v1.tsv')
    sample_qc_meta = hl.read_table('gs://hgdp_tgp/output/gnomad_v3.1_sample_qc_metadata_hgdp_tgp_subset.ht')
    var_meta = hl.read_table(
        'gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/gnomad_v3_sites_filters_only.ht')
    dense_mt = hl.read_matrix_table(
        'gs://gcp-public-data--gnomad/release/3.1.2/mt/genomes/gnomad.genomes.v3.1.2.hgdp_1kg_subset_dense.mt')
    
    var_meta = var_meta.naive_coalesce(5000)
    dense_mt = dense_mt.naive_coalesce(5000)


    # Takes a list of dicts and converts it to a struct format (works with nested structs too)
    def dict_to_struct(d):
        fields = {}
        for k, v in d.items():
            if isinstance(v, dict):
                v = dict_to_struct(v)
            fields[k] = v
        return hl.struct(**fields)

    # un-flattening a hail table with nested structure
    # dict to hold struct names as well as nested field names
    d = {}

    # Getting the row field names
    row = sample_meta.row_value

    # returns a dict with the struct names as keys and their inner field names as values
    for name in row:
        def recur(dict_ref, split_name):
            if len(split_name) == 1:
                dict_ref[split_name[0]] = row[name]
                return
            existing = dict_ref.get(split_name[0])
            if existing is not None:
                assert isinstance(existing, dict), existing
                recur(existing, split_name[1:])
            else:
                existing = {}
                dict_ref[split_name[0]] = existing
                recur(existing, split_name[1:])
        recur(d, name.split('.'))

    # using the dict created from flattened struct, creating new structs now un-flattened
    sample_meta = sample_meta.select(**dict_to_struct(d))
    sample_meta = sample_meta.key_by('s')

    # grabbing the columns needed from Alicia's metadata
    new_meta = sample_meta.select(sample_meta.hgdp_tgp_meta, sample_meta.bergstrom)

    # creating a table with gnomAD sample metadata and HGDP metadata
    ht = sample_qc_meta.annotate(**new_meta[sample_qc_meta.s])

    # stripping 'v3.1::' from the names to match with the densified MT
    ht = ht.key_by(s=ht.s.replace("v3.1::", ""))

    # Using hl.annotate_cols() method to annotate the gnomAD variant QC metadata onto the matrix table
    mt = dense_mt.annotate_cols(**ht[dense_mt.s])

    # annotating preQC dataset with variant metadata
    mt = mt.annotate_rows(**var_meta[mt.locus, mt.alleles])

    print(f"sample_qc: {sample_qc}\nvariant_qc: {variant_qc}\nduplicate: {duplicate}\noutlier_removal: { outlier_removal}\nld_pruning: {ld_pruning}\nrel_unrel: {rel_unrel}")
    
    if default:
        print("Returning default preQC matrix table")
        # returns preQC dataset
        return mt
    
    if post_qc:
        print("Returning post sample and variant QC matrix table with duplicates and PCA outliers removed")
        sample_qc = True
        variant_qc = True
        duplicate = True
        outlier_removal = True
    
    if sample_qc:
        print("Running sample QC")
        # run data through sample QC
        # filtering samples to those who should pass gnomADs sample QC
        # this filters to only samples that passed gnomad sample QC hard filters
        mt = mt.filter_cols(~mt.sample_filters.hard_filtered)

        # annotating partially filtered dataset with variant metadata
        mt = mt.annotate_rows(**var_meta[mt.locus, mt.alleles])

    if variant_qc:
        print("Running variant QC")
        # run data through variant QC
        # Subsetting the variants in the dataset to only PASS variants (those which passed gnomAD's variant QC)
        # PASS variants are variants which have an entry in the filters field.
        # This field contains an array which contains a bool if any variant qc filter was failed
        # This is the last step in the QC process
        mt = mt.filter_rows(hl.len(mt.filters) != 0, keep=False)

    if duplicate:
        print("Removing any duplicate samples")
        # Removing any duplicates in the dataset using hl.distinct_by_col() which removes
        # columns with a duplicate column key. It keeps one column for each unique key.
        # after updating to the new dense_mt, this step is no longer necessary to run
        mt = mt.distinct_by_col()

    if outlier_removal:
        print("Removing PCA outliers")
        # remove PCA outliers and duplicates
        # reading in the PCA outlier list
        # To read in the PCA outlier list, first need to read the file in as a list
        # using hl.hadoop_open here which allows one to read in files into hail from Google cloud storage
        pca_outlier_path = 'gs://hgdp-1kg/hgdp_tgp/pca_outliers_v2.txt'
        with hl.utils.hadoop_open(pca_outlier_path) as file:
            outliers = [line.rstrip('\n') for line in file]

        # Using hl.literal here to convert the list from a python object to a hail expression so that it can be used
        # to filter out samples
        outliers_list = hl.literal(outliers)

        # Using the list of PCA outliers, using the ~ operator which is a negation operator and obtains the compliment
        # In this case the compliment is samples which are not contained in the pca outlier list
        mt = mt.filter_cols(~outliers_list.contains(mt['s']))

    if ld_pruning:
        print("Returning ld pruned post variant and sample QC matrix table pre PCA outlier removal ")
        # read in dataset which has additional variant filtering and ld pruning run
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/intermediate_files/filtered_n_pruned_output_updated.mt')

    if rel_unrel == "all":
        print("Returning ld pruned post sample and variant \
              QC matrix table pre PCA outlier removal with related & unrelated individuals")
        # need to check what steps this dataset has gone through, this is something to discuss with Mary
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/intermediate_files/filtered_n_pruned_output_updated.mt')

    elif rel_unrel == 'related':
        print("Returning post sample and variant QC matrix table \
              pre PCA outlier removal with only related individuals")
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        #   - pc_relate - filter to only related individuals
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/rel_updated.mt')

        
    elif rel_unrel == 'unrelated':
        print("Returning post QC matrix table with only unrelated individuals")
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        #   - pc_relate - filter to only unrelated individuals
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/unrel_updated.mt')
        
    # Calculating both variant and sample_qc metrics on the mt before returning
    # so the stats are up to date with the version being written out
    mt = hl.sample_qc(mt)
    mt = hl.variant_qc(mt)
    
    return mt

# 2. Read in Datasets and Annotate
<br>
<details>
<summary>More about the input dataset 
     <br>[click <span style="color:green">here</span> to expand] </summary>

    This input matrix table is a combination of 3 datasets: a harmonized sample metadata for the HGDP+1KG dataset, a gnomAD v3.1 sample qc metadata with samples that failed gnomAD QC filters flagged, and a densified HGDP+1KG matrix table.
</details>
<br>
<details>
<summary> More information on Hail methods and expressions
    <br>[click <span style="color:green">here</span> to expand] </summary>
<ul>
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_matrix_table"> More on  <i> read_matrix_table() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.count"> More on  <i> count() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_table"> More on  <i> read_table() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows"> More on  <i> annotate_rows() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.describe"> More on  <i> describe() </i></a></li>
    </ul>
</details>

[Back to Index](#Index)

In [ ]:
# read-in the matrix table (shortened as mt)
mt = read_qc(default=True)

# how many snps and samples are there? counts 
print('Num of snps and samples prior to any analysis = ' + str(mt.count())) # 211358784 snps & 4151 samples 

# read in variant QC metadata containing information on which variants passed/failed gnomAD QC filters
var_meta = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.1/ht/genomes/gnomad.genomes.v3.1.1.sites.ht')

# annotate variant QC metadata onto mt 
mt = mt.annotate_rows(**var_meta[mt.locus, mt.alleles]) 

# explore combined mt 
mt.describe()

# 3. Investigating gnomAD sample filters

<br>
<details>
<summary> Why are we doing this? 
    <br>[click <span style="color:green">here</span> to expand] </summary>
    
9 out of the 28 gnomAD sample filters were dropping huge numbers of ancestrally diverse individuals (mostly African (AFR) and Oceanian (OCE) populations). The filters use gnomAD’s principal component analysis (PCA) which is obtained from other samples to residualize the distribution of values from different populations and identify outliers. If there is an error and outliers are identified, the sample fails the filter. 
</details>
<br>
<details>
<summary> More information on Hail methods and expressions 
    <br>[click <span style="color:green">here</span> to expand] </summary>
<ul>
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_cols"> More on  <i> filter_cols() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.expr.SetExpression.html#hail.expr.SetExpression.difference"> More on  <i> difference() </i></a></li>

<li><a href=" https://hail.is/docs/0.2/hail.expr.CollectionExpression.html#hail.expr.CollectionExpression.length"> More on  <i> length() </i></a></li>
    </ul>
</details>

[Back to Index](#Index)

In [ ]:
# put the gnomAD qc filters in a set 
all_sample_filters = set(mt['sample_filters']) 

# select out the filters that are removing whole populations despite them passing all other gnomAD filters
# if a filter name starts with 'fail_', add it to a new set after removing 'fail_' from the name  
bad_sample_filters = {re.sub('fail_', '', x) for x in all_sample_filters if x.startswith('fail_')} 

# filter out the samples that passed all gnomad QC filters OR only failed the filters that were removing population wholly
mt_filt = mt.filter_cols(mt['sample_filters']['qc_metrics_filters'].difference(bad_sample_filters).length() == 0)

# how many samples were removed by the initial QC?
print('Num of samples before initial QC = ' + str(mt.count()[1])) # 4151
print('Num of samples after initial QC = ' + str(mt_filt.count()[1])) # 4120
print('Samples removed = ' + str(mt.count()[1] - mt_filt.count()[1])) # 31